该课讲解在状态空间过大或连续时，如何使用函数逼近方法进行值函数估计，并重点介绍经典的深度强化学习算法DQN。

---

# 第8课：值函数近似（Function Approximation for Value Functions）

---

## Part 1：例子 — 曲线拟合

在现实强化学习中，状态空间常常非常大甚至连续，不能用表格（Tabular）方式存储每个状态的值。

举例：

* 给定一组离散样本点 $(x_i, y_i)$，拟合函数 $f(x; \theta)$ 来近似 $y$
* 类比强化学习中，用参数化函数 $V(s; \theta)$ 近似状态值函数

---

## Part 2：原理 — 目标函数介绍

目标是最小化估计值与真实值之间的误差，比如平方误差：

$$
J(\theta) = \mathbb{E}_{s \sim d^\pi} \left[ \big( V^\pi(s) - V(s; \theta) \big)^2 \right]
$$

---

## Part 3：原理 — 优化算法和函数选择

* **函数形式**：线性函数、非线性函数（如神经网络）
* **优化方法**：梯度下降或随机梯度下降
* **目标函数的梯度**：

$$
\nabla_\theta J(\theta) = \mathbb{E} \left[ 2 \big( V^\pi(s) - V(s; \theta) \big) \nabla_\theta V(s; \theta) \right]
$$

---

## Part 4：原理 — 示例与分析

* 由于真实 $V^\pi(s)$ 通常未知，使用**TD目标**代替
* 更新形式（基于 TD(0)）：

$$
\delta_t = r_{t+1} + \gamma V(s_{t+1}; \theta) - V(s_t; \theta)
$$

$$
\theta \leftarrow \theta + \alpha \delta_t \nabla_\theta V(s_t; \theta)
$$

---

## Part 5：Sarsa 和 Q-learning 的函数逼近

* 类似更新可用于动作值函数 $Q(s,a; \theta)$
* Sarsa更新：

$$
\delta_t = r_{t+1} + \gamma Q(s_{t+1}, a_{t+1}; \theta) - Q(s_t, a_t; \theta)
$$

* Q-learning更新：

$$
\delta_t = r_{t+1} + \gamma \max_{a'} Q(s_{t+1}, a'; \theta) - Q(s_t, a_t; \theta)
$$

---

## Part 6：DQN — 基本原理

Deep Q-Network（DQN）使用深度神经网络逼近 $Q$ 函数

---

### 关键点：

* 网络参数 $\theta$ 直接输出 $Q(s,a; \theta)$
* 目标值用固定的目标网络参数 $\theta^-$ 计算：

$$
y_t = r_{t+1} + \gamma \max_{a'} Q(s_{t+1}, a'; \theta^-)
$$

* 损失函数：

$$
L(\theta) = \mathbb{E}_{(s,a,r,s')} \left[ \big( y_t - Q(s,a; \theta) \big)^2 \right]
$$

---

## Part 7：DQN — Experience Replay（经验回放）

* 存储智能体的经历 $(s,a,r,s')$ 于回放缓冲区
* 训练时从缓冲区随机采样小批量数据，打破数据相关性
* 提升训练稳定性和样本利用率

---

## Part 8：DQN — 代码与例子

```python
# 简要伪代码
Initialize replay memory D
Initialize Q-network with random weights θ
Initialize target network weights θ- = θ

for episode in range(M):
    Initialize state s
    for t in range(T):
        Choose action a from s using ε-greedy policy derived from Q(s,a; θ)
        Take action a, observe reward r and next state s'
        Store transition (s,a,r,s') in D
        Sample random minibatch from D
        For each sample:
            y = r + γ * max_a' Q(s', a'; θ-)
        Update θ by minimizing (y - Q(s,a; θ))^2
        Every C steps, update θ- = θ
        s = s'
```

---

## 本课总结

* 函数逼近是解决大规模或连续状态空间的关键
* TD目标和函数逼近结合，形成近似TD学习
* DQN利用深度神经网络和经验回放，极大提升了Q-learning的表现

